In [ ]:
import importlib
import pandas as pd
import helpers

from pathlib import Path

importlib.reload(helpers)

from helpers import REGIJE_FILTER, Impute_missing

In [ ]:
import locale
from pandas.tseries.offsets import MonthEnd

locale.setlocale(locale.LC_ALL, "sl_SI.utf8")
WIDE_PATH = Path("podatki/wide_df.csv")


def get_last_day_of_month(row):
    return pd.to_datetime(f"{row['Leto']}-{row['Mesec']}", format="%Y-%b") + MonthEnd(0)


df = pd.read_csv(WIDE_PATH)
df.insert(loc=2, column="Datum", value=df.apply(get_last_day_of_month, axis=1))
df.drop(columns=['Postaja','Leto','Mesec'],inplace=True)
df

,Regija,Datum,PM10,SO2,NO2,NOX
0,Savinjska,1997-01-31,NaN,72.0,42.0,70.0
1,Zasavska,1997-01-31,NaN,196.0,NaN,NaN
2,Savinjska,1997-01-31,NaN,66.0,50.0,NaN
3,Posavska,1997-01-31,NaN,41.0,NaN,NaN
4,Koroška,1997-01-31,NaN,71.0,NaN,NaN
...,...,...,...,...,...,...
10711,Savinjska,2023-12-31,NaN,5.0,NaN,NaN
10712,Osrednjeslovenska,2023-12-31,26.0,4.0,27.0,67.0
10713,Zasavska,2023-12-31,26.0,0.0,22.0,63.0
10714,Savinjska,2023-12-31,NaN,6.0,6.0,7.0


In [ ]:
def aggregate_by_snov(df: pd.DataFrame, snov="") -> pd.DataFrame:
    df = df.copy()
    df_aggregated = df.groupby(["Regija", "Datum"])[snov].mean().reset_index()
    df_aggregated.sort_values(by=["Regija", "Datum"])
    df_aggregated = df_aggregated[df_aggregated["Regija"].isin(REGIJE_FILTER[snov])]

    df_pivoted = df_aggregated.pivot(index="Datum", columns="Regija", values=snov)
    df_filled = Impute_missing(df_pivoted)

    return df_filled


df_so2 = aggregate_by_snov(df, "SO2")

df.sample(8)



,Postaja,Regija,Datum,Leto,Mesec,PM10,SO2,NO2,NOX
2478,trbovlje,Zasavska,2005-03-31,2005,Mar,74.0,25.0,33.0,46.0
9812,novo mesto,Jugovzhodna Slovenija,2022-03-31,2022,Mar,28.0,NaN,15.0,18.0
9027,gorenje polje,Goriška,2020-10-31,2020,Okt,14.0,NaN,NaN,NaN
8169,lj center,Osrednjeslovenska,2018-11-30,2018,Nov,31.0,2.0,36.0,109.0
10122,mb titova,Podravska,2022-11-30,2022,Nov,22.0,NaN,23.0,63.0
10253,morsko,Goriška,2023-02-28,2023,Feb,30.0,NaN,NaN,NaN
2113,eis krsko,Posavska,2004-03-31,2004,Mar,NaN,37.0,NaN,NaN
10442,spuhlja,Podravska,2023-06-30,2023,Jun,19.0,NaN,NaN,NaN
